In [11]:
# ==================================================
# HIGH-PRECISION NUMERICAL INTEGRATION (SIMPLIFIED)
# ==================================================
import numpy as np
from scipy import integrate

# ----------------------------
# Problem 1: ∫0^∞ 1/(1 + 25 x²) dx
# ----------------------------
analytical_1 = np.pi / 10
print("Problem 1: ∫0^∞ 1/(1 + 25 x²) dx")
print(f"Analytical: {analytical_1:.15f}")

# Transform x = (1/5)*tan(t) => integrand becomes constant 1/5
def f1_transformed(t):
    return 1/5

# SciPy quad
res1_quad, err1_quad = integrate.quad(f1_transformed, 0, np.pi/2, epsabs=1e-15, epsrel=1e-15)
print(f"SciPy quad: {res1_quad:.15f} (est. error: {err1_quad:.2e})")

# Gauss-Legendre
def gauss_legendre(f, a, b, n=16):
    result, _ = integrate.fixed_quad(f, a, b, n=n)
    return result

res1_gauss = gauss_legendre(f1_transformed, 0, np.pi/2, n=16)
print(f"Gauss-Legendre (n=16): {res1_gauss:.15f} (error: {abs(res1_gauss-analytical_1):.2e})")

# ----------------------------
# Problem 2: ∫0^1 ln(x)/(1 + 25 x²) dx
# ----------------------------
print("\nProblem 2: ∫0^1 ln(x)/(1 + 25 x²) dx")

# Original integrand
def f2(x):
    x = np.atleast_1d(x)
    result = np.zeros_like(x, dtype=float)
    mask = x > 0
    result[mask] = np.log(x[mask]) / (1 + 25 * x[mask]**2)
    return result if result.size > 1 else result.item()

# Method 1: SciPy quad
res2_quad, err2_quad = integrate.quad(f2, 0, 1, epsabs=1e-15, epsrel=1e-15)
print(f"SciPy quad: {res2_quad:.15f} (est. error: {err2_quad:.2e})")

# Method 2: Variable substitution x = t^k to smooth ln(x) singularity
def f2_transformed(t, k=4):
    t = np.atleast_1d(t)
    result = np.zeros_like(t, dtype=float)
    mask = t > 0
    result[mask] = k**2 * t[mask]**(k-1) * np.log(t[mask]) / (1 + 25 * t[mask]**(2*k))
    return result if result.size > 1 else result.item()

res2_gauss = gauss_legendre(lambda t: f2_transformed(t, k=4), 0, 1, n=128)
print(f"Gauss-Legendre (transformed, n=128): {res2_gauss:.15f} (error: {abs(res2_gauss-res2_quad):.2e})")

# ----------------------------
# Final verification
# ----------------------------
target_precision = 1e-10
print("\nVerification:")
print(f"Problem 1 error < {target_precision}: {abs(res1_gauss-analytical_1) < target_precision}")
print(f"Problem 2 difference between methods < {target_precision}: {abs(res2_gauss-res2_quad) < target_precision}")


Problem 1: ∫0^∞ 1/(1 + 25 x²) dx
Analytical: 0.314159265358979
SciPy quad: 0.314159265358979 (est. error: 3.49e-15)
Gauss-Legendre (n=16): 0.314159265358979 (error: 0.00e+00)

Problem 2: ∫0^1 ln(x)/(1 + 25 x²) dx
SciPy quad: -0.545444563419798 (est. error: 6.75e-15)
Gauss-Legendre (transformed, n=128): -0.545444563419786 (error: 1.19e-14)

Verification:
Problem 1 error < 1e-10: True
Problem 2 difference between methods < 1e-10: True
